# Homework 6: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [10]:
library(dplyr)

lexical <- read.csv("LexicalData.csv")
items <- read.csv("Items.csv")


lexical$D_RT_nocomma <- gsub(",","",lexical$D_RT)
lexical$D_RT_nocomma <- as.numeric(lexical$D_RT_nocomma)

items.small <- items %>% mutate(D_Word = Word) %>% select(D_Word, Length, Log_Freq_HAL)
 

data <- left_join(items.small, lexical, by = "D_Word")

---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [18]:
linear.model <- lm(D_RT_nocomma ~ Log_Freq_HAL + Length + Log_Freq_HAL*Length, data = data)
summary(linear.model)


Call:
lm(formula = D_RT_nocomma ~ Log_Freq_HAL + Length + Log_Freq_HAL * 
    Length, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library. 

In [14]:
# dont need to install because I already have it
library(lme4)

Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [19]:
MLM.1 = lmer(D_RT_nocomma ~ Log_Freq_HAL + Length + Log_Freq_HAL*Length + (1 | Sub_ID), data=data) 
summary(MLM.1)


Linear mixed model fit by REML ['lmerMod']
Formula: D_RT_nocomma ~ Log_Freq_HAL + Length + Log_Freq_HAL * Length +  
    (1 | Sub_ID)
   Data: data

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> *3/4 t-values in the MLM have decreased compared to those produced by the linear regression. This reflects the fact that the MLM produces a more conservative model. The MLM accounts for random variance in the intercept, which decreases the varience unaccounted for in the outcome (reaction time). * 

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [1]:
ic = AIC(linear.model, MLM.1)
ic
diff(ic$AIC)

ERROR: Error in AIC(linear.model, MLM.1): object 'linear.model' not found


> *Comparing AIC values between the linear regression model fit and that of the multilevel model demonstrates a smaller AIC for the MLM. The multilevel model accounts for substantially more variance in reaction time than the simple linear model, even after accounting for increased complexity * 

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

> *There are a couple of other random effects that could be controlled for in this data set. You could account for the random slope of each predictor (length and Freq_HAL) to see if people vary in their relationships between the predictors and reaction time.* 
> 

**DUE:** 5pm EST, March 25, 2022

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Delainey Wescott*